<a href="https://colab.research.google.com/github/ericrosso/BitcoinPrediction/blob/main/Bitcoin_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Prima di iniziare: Scarico il dataset da google drive con la libreria gdown

In [ ]:
!gdown 1u4LyWQvDoWx-Eozmu8dr_X8mHdSzOjl2

Downloading...
From: https://drive.google.com/uc?id=1u4LyWQvDoWx-Eozmu8dr_X8mHdSzOjl2
To: /content/bitcoin_data.csv
100% 231k/231k [00:00<00:00, 87.2MB/s]


In [ ]:
!gdown 15SB2_PIKYoCqDfx7LattasoWl5LJEJWY

Downloading...
From: https://drive.google.com/uc?id=15SB2_PIKYoCqDfx7LattasoWl5LJEJWY
To: /content/readnews.csv
100% 312/312 [00:00<00:00, 1.73MB/s]


PRIMO STEP: importo tutte le librerie necessarie al funzionamento del codice

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
import time
import datetime as dt
from pandas import read_csv
from matplotlib import pyplot
from pandas import DataFrame
from numpy import array
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

SECONDO STEP: leggo il dataset da csv e lo salvo in "dataset"

In [ ]:
dataset = pd.read_csv("bitcoin_data.csv", header=0, index_col=0, parse_dates=False, squeeze=True);

<ipython-input-14-37a6f34daa94>:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  dataset = pd.read_csv("bitcoin_data.csv", header=0, index_col=0, parse_dates=False, squeeze=True);


Controllo che il dataset sia stato correttamente caricato

In [ ]:
dataset.head(20)

In [ ]:
dataset.info()

TERZO STEP: Creo una funzione per convertire "dataset" in un formato adeguato al modello scelto.
Alcuni step sono:

*   Converto il "dataset" in un formato DataFrame (struttura dati di pandas)
*   Etichetto le colonne con il nome della colonna
*   Rimuovo le colonne con valori nulli



In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
 """
 Frame a time series as a supervised learning dataset.
 Arguments:
 data: Sequence of observations as a list or NumPy array.
 n_in: Number of lag observations as input (X).
 n_out: Number of observations as output (y).
 dropnan: Boolean whether or not to drop rows with NaN values.
 Returns:
 Pandas DataFrame of series framed for supervised learning.
 """
 n_vars = 1 if type(data) is list else data.shape[1]
 df = DataFrame(data)
 cols, names = list(), list()
 # input sequence (t-n, ... t-1)
 for i in range(n_in, 0, -1):
   cols.append(df.shift(i))
 names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
 # forecast sequence (t, t+1, ... t+n)
 for i in range(0, n_out):
   cols.append(df.shift(-i))
 if i == 0:
   names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
 else:
   names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
 # put it all together
 agg = pd.concat(cols, axis=1)
 agg.columns = names
 print(agg)
 # drop rows with NaN values
 if dropnan:
   agg.dropna(inplace=True)
 return agg

Eseguo la funzione e salvo il risultato sulla variabile "values"

In [ ]:
values = series_to_supervised(dataset.values, n_in=1, n_out=1, dropnan=True).values

Imposto la grandezza del dataset per il training e il dataset per il testing (70%,30%)

In [ ]:
train_size = int(len(values) * 0.7)
train, test = values[:train_size], values[train_size:]

Tolgo la colonna da prevedere da parte del modello

In [ ]:
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
print(train_X)

Questo reshape cambia il formato dei dati in modo che la data metta le righe in modo sequenziale e non venga considerato come parametro

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
print(train_X)
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

Dichiaro il modello LTSM (Long Short-Term Memory)[prima riga]  , aggiungo un livello LSTM alla rete neurale  con il dataset finale[seconda e terza riga](128 e 64 rappresentano il nr di unità dell'LSTM). l'ultima riga invece restituirà il valore finale di output. 

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1))

Compilo il modello e lo faccio eseguire per il modello fit

In [ ]:
model.compile(loss='mae', optimizer='adam')
history = model.fit(train_X, train_y, epochs=170, batch_size=40, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Modello precaricato (prima deve essere caricato da gdown)

In [ ]:
model = keras.models.load_model('bitcoinmodel.h5')

Previsione dei valori da parte del modello

In [ ]:
data = pd.DataFrame(np.array([[51500.05703,55556.80394,50416.72833,55385.44519,77199761550,9.96E+11,45695.36703,46962.64088,45060.13319,77.58690461], [55315.91067,55315.91067,53519.69323,53809.13817,88802412237,1.02E+12,45980.74264,47143.80909,45038.1895,70.64797873]]),
                   columns=['Open','High','Low','Close','Volume','Market Cap','MA(20)','MA(50)','MA(200)','RSI(14)(SMA)'])
valuesdata = series_to_supervised(data.values, n_in=1, n_out=1, dropnan=True).values
print(valuesdata)
train_X, train_y = valuesdata[:, :-1], valuesdata[:, -1]
#print(train_X)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
#print(train_X)
new_output = model.predict(train_X)
print(new_output[0,0])

Salvo il modello in locale

In [ ]:
model.save("bitcoinmodel.h5")

Grafico per visualizzare la perdita media per ogni epoca

In [ ]:
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()